####  Adding features incrementally to observe the impact on accuracy to determine at which point the accuracy stops improving. This approach is commonly known as feature selection or feature engineering.

#### Adding features one by one and monitoring the accuracy helps in assessing the contribution of each feature to the predictive power of your model. This process helps identify the most informative features and can prevent overfitting by avoiding the inclusion of irrelevant or redundant features.

In [1]:
#Import all the necessary libraries
#Import all the necessary libraries
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as nm
nm.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
import time

path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename1='Common_Data.xlsx'
filename2='Intersection_Data.xlsx'
#Access all the necessary variables and datasets
%store -r top_features_data
%store -r random_forest_data
random_forest_data.shape #161 statistical features and one output column
# Using the top_features_data dataframe created in random_Forest_Code which has columns for top 20 features and also output
#Usibg the appended_data dataframe created in Random_Forest_Code


GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


(591, 162)

### Training the base model 

In [2]:
top_features_data = shuffle(top_features_data, random_state=42)
#Defining train and test sizes
total_length= top_features_data.shape[0]
train_size=int(top_features_data.shape[0]*0.8)
test_size=top_features_data.shape[0]-train_size

# First converting dataframe series into numpy array and setting the datatype as float32 for all data
# In second step converting all 1D array into 2D array
train_X=top_features_data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

train_Y=top_features_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

test_X=top_features_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

test_Y=top_features_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

print('Dimension of training data',train_X.shape)
print('Dimension of testing data',test_X.shape)

#print(train_X)
print(type(train_X))
print(train_Y.shape)
test_Y.shape #2-D array
#train_Y

Dimension of training data (472, 20)
Dimension of testing data (119, 20)
<class 'numpy.ndarray'>
(472, 1)


(119, 1)

In [3]:
scaler=RobustScaler() # scaler object

# First fit the scaler with your training data.
# It will find those parameters using which it is going to scale.
# Then transform any data you want.

train_X=scaler.fit_transform(train_X) 

# Since it has scaled  the data as per the parameters it learnt during training data.
# So it will require 2D array/similiar dimension as of training data.
# So testing data should be a 2D array.
# Testing data only need to be transformed on the basis of the parameters learnt using fit function on training data.

test_X=scaler.transform(test_X) #Scaling the training data and using the same parameters for testing data
#test
#train_X[:]

In [4]:
from keras.regularizers import l2
#Creating a Model
print('Total No of features',len(train_X[0,:])) #No of features
base_model=Sequential() 

base_model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
base_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
base_model.summary()

Total No of features 20
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                1050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0
_________________________________________________________________


In [5]:
#model.compile(optimizer='_' , loss='_' , metrics=['accuracy'])
base_model.compile(loss='binary_crossentropy',optimizer=SGD(momentum=0.9),metrics=['accuracy'])

In [6]:
#Fitting the training data
#base_model.fit(training data input, training data output, batch_size=_,validation_data=(x_test,y_test))
base_model.fit(train_X,train_Y,epochs=150,batch_size=len(train_X),validation_data=(test_X,test_Y))

# Epochs is number of times i want to do forward and backward propogation.
# batch size is used for update the weights in mini batches. The default back size is 32.
# Validation data : accuracy looking like on validation data after every epoch, you can give test data here too. 
#The loss will be reduced due to backpropgation through time.

#Add regularization or increase number of epochs for better accuracy.

Epoch 1/150
1/1 [==============================] - 1s 799ms/step - loss: 10.9257 - accuracy: 0.5042 - val_loss: 9.0843 - val_accuracy: 0.4790
Epoch 2/150
1/1 [==============================] - 0s 32ms/step - loss: 11.7505 - accuracy: 0.5212 - val_loss: 9.3289 - val_accuracy: 0.4622
Epoch 3/150
1/1 [==============================] - 0s 51ms/step - loss: 13.8135 - accuracy: 0.5254 - val_loss: 6.2027 - val_accuracy: 0.4538
Epoch 4/150
1/1 [==============================] - 0s 63ms/step - loss: 8.0377 - accuracy: 0.5127 - val_loss: 3.1773 - val_accuracy: 0.4370
Epoch 5/150
1/1 [==============================] - 0s 47ms/step - loss: 2.9813 - accuracy: 0.4767 - val_loss: 2.6635 - val_accuracy: 0.4286
Epoch 6/150
1/1 [==============================] - 0s 47ms/step - loss: 2.4421 - accuracy: 0.4640 - val_loss: 1.9813 - val_accuracy: 0.4790
Epoch 7/150
1/1 [==============================] - 0s 47ms/step - loss: 1.8508 - accuracy: 0.4746 - val_loss: 1.7329 - val_accuracy: 0.5126
Epoch 8/150
1/1 

1/1 [==============================] - 0s 42ms/step - loss: 1.0878 - accuracy: 0.6250 - val_loss: 1.1827 - val_accuracy: 0.5798
Epoch 60/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0725 - accuracy: 0.6208 - val_loss: 1.1660 - val_accuracy: 0.5798
Epoch 61/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0710 - accuracy: 0.6229 - val_loss: 1.1614 - val_accuracy: 0.5714
Epoch 62/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0665 - accuracy: 0.6314 - val_loss: 1.1628 - val_accuracy: 0.5462
Epoch 63/150
1/1 [==============================] - 0s 47ms/step - loss: 1.0592 - accuracy: 0.6271 - val_loss: 1.2445 - val_accuracy: 0.5462
Epoch 64/150
1/1 [==============================] - 0s 31ms/step - loss: 1.0605 - accuracy: 0.6419 - val_loss: 1.1537 - val_accuracy: 0.5546
Epoch 65/150
1/1 [==============================] - 0s 32ms/step - loss: 1.0508 - accuracy: 0.6462 - val_loss: 1.1457 - val_accuracy: 0.5462
Epoch 66/150
1/1 [========

1/1 [==============================] - 0s 79ms/step - loss: 0.9354 - accuracy: 0.7013 - val_loss: 1.0267 - val_accuracy: 0.5630
Epoch 118/150
1/1 [==============================] - 0s 80ms/step - loss: 0.9314 - accuracy: 0.7055 - val_loss: 1.0704 - val_accuracy: 0.5630
Epoch 119/150
1/1 [==============================] - 0s 86ms/step - loss: 0.9327 - accuracy: 0.7055 - val_loss: 1.0235 - val_accuracy: 0.5630
Epoch 120/150
1/1 [==============================] - 0s 70ms/step - loss: 0.9292 - accuracy: 0.7034 - val_loss: 1.0217 - val_accuracy: 0.5714
Epoch 121/150
1/1 [==============================] - 0s 79ms/step - loss: 0.9299 - accuracy: 0.7034 - val_loss: 1.0206 - val_accuracy: 0.5714
Epoch 122/150
1/1 [==============================] - 0s 63ms/step - loss: 0.9258 - accuracy: 0.7055 - val_loss: 1.0578 - val_accuracy: 0.5714
Epoch 123/150
1/1 [==============================] - 0s 62ms/step - loss: 0.9258 - accuracy: 0.7055 - val_loss: 1.0177 - val_accuracy: 0.5714
Epoch 124/150
1/1 [=

In [11]:
# Create a list to store the selected features and their corresponding accuracies
base_features = top_features_data.columns.tolist()[:-1]
base_features

['STEELLEVELINMOLD-SETMEAN',
 'NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDWIDTHTOPMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG9+10+11+12TOPSECONDARYCOOLINGWATERFLOWACTUALVARIANCE',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'BROADFACE1WATERFLOWMEAN',
 'NARROWFACE1WATERFLOWMEAN',
 'PER_SIMEAN',
 'PER_MNMEAN',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'LIQUIDUSTEMPMEAN',
 'STRANDDRIVECURRENTMAX',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'BROADFACE2WATERFLOWVARIANCE',
 'SEG5+6TOPSECONDARYCOOLINGWATERFLOWACTUALVARIANCE',
 'SEG5+6BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN']

In [9]:
# Evaluate the model

testPredict_probs=base_model.predict(test_X)
#testPredict_probs

# Apply threshold to obtain predicted classes
threshold = 0.5  # Adjust this threshold as needed
y_pred = (testPredict_probs > threshold).astype(int)
y_pred

base_accuracy = accuracy_score(test_Y,y_pred)
base_accuracy

4/4 [==============================] - 0s 1ms/step


0.6470588235294118

### Data for new model

In [11]:
%store -r feature_imp_RF
total_features=feature_imp_RF["Feature"]
total_features_list= total_features.tolist()

remaining_features=[]
for feature in total_features_list:
    if feature not in base_features:
        remaining_features.append(feature) 
        
len(remaining_features) #161-20 features

141

### Iterating over remaining features

In [14]:
import time
new_features=[]
count=0
optimal_features=[]
accuracy_store=[base_accuracy]

# Start the timer
start_time = time.time()

for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]
    
    #Creating a new data
    new_data=random_forest_data.loc[:,new_features]
    new_data = shuffle(new_data, random_state=42)
    
    #Splitting the new_data into train and test
    new_total_length= new_data.shape[0]
    new_train_size=int(new_data.shape[0]*0.8)
    new_test_size=new_data.shape[0]-train_size
    new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
    new_train_Y=top_features_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
    new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
    new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
    new_test_Y=top_features_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
    new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array}

    
    # Create a new model with the updated feature set
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
    # Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    print(count)
    print(feature)
    
    
    # Train the new model
    new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))
    
    # Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    accuracy_store.append(new_accuracy)
    
    # Update the baseline feature set if the accuracy score improved
    if new_accuracy> base_accuracy:
        base_features = new_features
        base_accuracy = new_accuracy
        optimal_features.append(new_features)
    
    # Check for convergence
    #if len(base_features) == random_forest_data.iloc[:,:-1].shape[1]:
     #   break
        
# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

0
PER_SIMEAN
Epoch 1/150
95/95 [==============================] - 1s 2ms/step - loss: 13.9628 - accuracy: 0.4936 - val_loss: 26.8293 - val_accuracy: 0.6050
Epoch 2/150
95/95 [==============================] - 0s 1ms/step - loss: 8.8748 - accuracy: 0.5445 - val_loss: 20.8366 - val_accuracy: 0.5210
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 6.6632 - accuracy: 0.5508 - val_loss: 14.7232 - val_accuracy: 0.4958
Epoch 4/150
95/95 [==============================] - 0s 1ms/step - loss: 5.2312 - accuracy: 0.5847 - val_loss: 4.4693 - val_accuracy: 0.5882
Epoch 5/150
95/95 [==============================] - 0s 1ms/step - loss: 3.0383 - accuracy: 0.5953 - val_loss: 5.9051 - val_accuracy: 0.5966
Epoch 6/150
95/95 [==============================] - 0s 1ms/step - loss: 2.3748 - accuracy: 0.5869 - val_loss: 5.1047 - val_accuracy: 0.5630
Epoch 7/150
95/95 [==============================] - 0s 1ms/step - loss: 2.2844 - accuracy: 0.5487 - val_loss: 4.2178 - val_accuracy: 0.4

Epoch 116/150
95/95 [==============================] - 0s 2ms/step - loss: 1.0461 - accuracy: 0.6377 - val_loss: 1.3711 - val_accuracy: 0.6555
Epoch 117/150
95/95 [==============================] - 0s 2ms/step - loss: 1.6622 - accuracy: 0.5869 - val_loss: 2.7155 - val_accuracy: 0.5714
Epoch 118/150
95/95 [==============================] - 0s 2ms/step - loss: 1.0782 - accuracy: 0.6758 - val_loss: 1.4414 - val_accuracy: 0.6387
Epoch 119/150
95/95 [==============================] - 0s 2ms/step - loss: 1.2079 - accuracy: 0.6377 - val_loss: 1.9861 - val_accuracy: 0.5210
Epoch 120/150
95/95 [==============================] - 0s 2ms/step - loss: 1.0773 - accuracy: 0.6716 - val_loss: 1.4916 - val_accuracy: 0.6723
Epoch 121/150
95/95 [==============================] - 0s 2ms/step - loss: 1.2676 - accuracy: 0.6568 - val_loss: 1.4526 - val_accuracy: 0.6303
Epoch 122/150
95/95 [==============================] - 0s 2ms/step - loss: 1.2327 - accuracy: 0.6081 - val_loss: 1.5386 - val_accuracy: 0.6387

Epoch 23/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5592 - accuracy: 0.5657 - val_loss: 1.6109 - val_accuracy: 0.5210
Epoch 24/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3432 - accuracy: 0.6102 - val_loss: 1.3742 - val_accuracy: 0.5378
Epoch 25/150
95/95 [==============================] - 0s 1ms/step - loss: 1.7853 - accuracy: 0.5530 - val_loss: 1.8686 - val_accuracy: 0.5042
Epoch 26/150
95/95 [==============================] - 0s 1ms/step - loss: 1.9926 - accuracy: 0.5636 - val_loss: 1.9301 - val_accuracy: 0.5210
Epoch 27/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5093 - accuracy: 0.5530 - val_loss: 2.3290 - val_accuracy: 0.4622
Epoch 28/150
95/95 [==============================] - 0s 2ms/step - loss: 1.4341 - accuracy: 0.5275 - val_loss: 1.2434 - val_accuracy: 0.5714
Epoch 29/150
95/95 [==============================] - 0s 1ms/step - loss: 1.6096 - accuracy: 0.5424 - val_loss: 1.4825 - val_accuracy: 0.5882
Epoch 

Epoch 81/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5040 - accuracy: 0.5847 - val_loss: 1.3699 - val_accuracy: 0.6134
Epoch 82/150
95/95 [==============================] - 0s 1ms/step - loss: 0.9567 - accuracy: 0.6017 - val_loss: 1.9739 - val_accuracy: 0.5546
Epoch 83/150
95/95 [==============================] - 0s 2ms/step - loss: 1.6828 - accuracy: 0.5551 - val_loss: 1.6370 - val_accuracy: 0.6218
Epoch 84/150
95/95 [==============================] - 0s 2ms/step - loss: 0.9847 - accuracy: 0.6271 - val_loss: 1.3526 - val_accuracy: 0.5966
Epoch 85/150
95/95 [==============================] - 0s 2ms/step - loss: 1.1336 - accuracy: 0.6017 - val_loss: 3.0591 - val_accuracy: 0.4622
Epoch 86/150
95/95 [==============================] - 0s 1ms/step - loss: 1.3179 - accuracy: 0.5975 - val_loss: 1.5613 - val_accuracy: 0.5546
Epoch 87/150
95/95 [==============================] - 0s 1ms/step - loss: 1.2040 - accuracy: 0.6165 - val_loss: 2.2042 - val_accuracy: 0.4958
Epoch 

95/95 [==============================] - 0s 2ms/step - loss: 2.0029 - accuracy: 0.6059 - val_loss: 4.8637 - val_accuracy: 0.5546
Epoch 46/150
95/95 [==============================] - 0s 1ms/step - loss: 2.1147 - accuracy: 0.6165 - val_loss: 1.7573 - val_accuracy: 0.6134
Epoch 47/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5066 - accuracy: 0.6271 - val_loss: 2.2867 - val_accuracy: 0.6387
Epoch 48/150
95/95 [==============================] - 0s 1ms/step - loss: 1.8560 - accuracy: 0.5869 - val_loss: 2.4710 - val_accuracy: 0.5546
Epoch 49/150
95/95 [==============================] - 0s 1ms/step - loss: 2.5421 - accuracy: 0.5678 - val_loss: 2.6292 - val_accuracy: 0.5378
Epoch 50/150
95/95 [==============================] - 0s 1ms/step - loss: 1.5830 - accuracy: 0.5593 - val_loss: 3.0482 - val_accuracy: 0.5714
Epoch 51/150
95/95 [==============================] - 0s 1ms/step - loss: 1.8799 - accuracy: 0.6059 - val_loss: 1.4014 - val_accuracy: 0.6555
Epoch 52/150
95/95 

...

Epoch 60/150
95/95 [==============================] - 0s 3ms/step - loss: 2.6457 - accuracy: 0.5953 - val_loss: 2.4968 - val_accuracy: 0.6218
Epoch 61/150
95/95 [==============================] - 0s 3ms/step - loss: 2.7361 - accuracy: 0.5784 - val_loss: 6.0047 - val_accuracy: 0.5462
Epoch 62/150
95/95 [==============================] - 0s 3ms/step - loss: 2.9532 - accuracy: 0.6017 - val_loss: 3.6097 - val_accuracy: 0.5210
Epoch 63/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6587 - accuracy: 0.6229 - val_loss: 4.2251 - val_accuracy: 0.5462
Epoch 64/150
95/95 [==============================] - 0s 3ms/step - loss: 2.9754 - accuracy: 0.5890 - val_loss: 2.8028 - val_accuracy: 0.6387
Epoch 65/150
95/95 [==============================] - 0s 3ms/step - loss: 2.8001 - accuracy: 0.6165 - val_loss: 2.5502 - val_accuracy: 0.6975
Epoch 66/150
95/95 [==============================] - 0s 3ms/step - loss: 3.5252 - accuracy: 0.6144 - val_loss: 2.2130 - val_accuracy: 0.6891
Epoch 

Epoch 24/150
95/95 [==============================] - 0s 3ms/step - loss: 2.2951 - accuracy: 0.6017 - val_loss: 5.2166 - val_accuracy: 0.5378
Epoch 25/150
95/95 [==============================] - 0s 3ms/step - loss: 2.1309 - accuracy: 0.6229 - val_loss: 2.0187 - val_accuracy: 0.5546
Epoch 26/150
95/95 [==============================] - 0s 3ms/step - loss: 3.9963 - accuracy: 0.5487 - val_loss: 3.3297 - val_accuracy: 0.5714
Epoch 27/150
95/95 [==============================] - 0s 3ms/step - loss: 2.4450 - accuracy: 0.6144 - val_loss: 3.0421 - val_accuracy: 0.5462
Epoch 28/150
95/95 [==============================] - 0s 3ms/step - loss: 2.1434 - accuracy: 0.5826 - val_loss: 2.3109 - val_accuracy: 0.6134
Epoch 29/150
95/95 [==============================] - 0s 4ms/step - loss: 1.9194 - accuracy: 0.5932 - val_loss: 3.0293 - val_accuracy: 0.5462
Epoch 30/150
95/95 [==============================] - 0s 4ms/step - loss: 3.5211 - accuracy: 0.5572 - val_loss: 5.2115 - val_accuracy: 0.5210
Epoch 

Epoch 82/150
95/95 [==============================] - 0s 4ms/step - loss: 3.0508 - accuracy: 0.5996 - val_loss: 2.4286 - val_accuracy: 0.6387
Epoch 83/150
95/95 [==============================] - 0s 4ms/step - loss: 2.8333 - accuracy: 0.5911 - val_loss: 3.6988 - val_accuracy: 0.5210
Epoch 84/150
95/95 [==============================] - 0s 4ms/step - loss: 3.0140 - accuracy: 0.6059 - val_loss: 3.1723 - val_accuracy: 0.5798
Epoch 85/150
95/95 [==============================] - 0s 4ms/step - loss: 2.7601 - accuracy: 0.6208 - val_loss: 2.9178 - val_accuracy: 0.5630
Epoch 86/150
95/95 [==============================] - 0s 4ms/step - loss: 1.8640 - accuracy: 0.6292 - val_loss: 4.0764 - val_accuracy: 0.5546
Epoch 87/150
95/95 [==============================] - 0s 4ms/step - loss: 1.8765 - accuracy: 0.6483 - val_loss: 2.3115 - val_accuracy: 0.5546
Epoch 88/150
95/95 [==============================] - 0s 3ms/step - loss: 1.8703 - accuracy: 0.6419 - val_loss: 2.5289 - val_accuracy: 0.6387
Epoch 

95/95 [==============================] - 0s 3ms/step - loss: 2.7964 - accuracy: 0.5869 - val_loss: 3.2221 - val_accuracy: 0.5294
Epoch 47/150
95/95 [==============================] - 0s 3ms/step - loss: 1.1500 - accuracy: 0.6716 - val_loss: 5.1045 - val_accuracy: 0.4538
Epoch 48/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3790 - accuracy: 0.6631 - val_loss: 4.8740 - val_accuracy: 0.4622
Epoch 49/150
95/95 [==============================] - 0s 3ms/step - loss: 1.8047 - accuracy: 0.6081 - val_loss: 1.8295 - val_accuracy: 0.6134
Epoch 50/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6338 - accuracy: 0.5975 - val_loss: 1.3647 - val_accuracy: 0.6303
Epoch 51/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3662 - accuracy: 0.6547 - val_loss: 1.3406 - val_accuracy: 0.6807
Epoch 52/150
95/95 [==============================] - 0s 3ms/step - loss: 1.6936 - accuracy: 0.6271 - val_loss: 1.5723 - val_accuracy: 0.6555
Epoch 53/150
95/95 

Epoch 104/150
95/95 [==============================] - 0s 3ms/step - loss: 1.7704 - accuracy: 0.6208 - val_loss: 3.9749 - val_accuracy: 0.5462
Epoch 105/150
95/95 [==============================] - 0s 3ms/step - loss: 1.4289 - accuracy: 0.6229 - val_loss: 1.5284 - val_accuracy: 0.6471
Epoch 106/150
95/95 [==============================] - 0s 3ms/step - loss: 1.0172 - accuracy: 0.6589 - val_loss: 1.4735 - val_accuracy: 0.6639
Epoch 107/150
95/95 [==============================] - 0s 3ms/step - loss: 1.3407 - accuracy: 0.6377 - val_loss: 2.4577 - val_accuracy: 0.4706
Epoch 108/150
95/95 [==============================] - 0s 3ms/step - loss: 1.1968 - accuracy: 0.6377 - val_loss: 1.4464 - val_accuracy: 0.6218
Epoch 109/150
95/95 [==============================] - 0s 3ms/step - loss: 1.5716 - accuracy: 0.6059 - val_loss: 3.4302 - val_accuracy: 0.4706
Epoch 110/150
95/95 [==============================] - 0s 3ms/step - loss: 1.9946 - accuracy: 0.5784 - val_loss: 1.6626 - val_accuracy: 0.6134

In [16]:
print("Simulation ended",training_time)
print("Best feature set:", base_features)
print("Best accuracy score:", base_accuracy)

Simulation ended 5388.467425107956
Best feature set: ['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'STEELLEVELINMOLD-SETMEAN', 'MOLDWIDTHTOPMEAN', 'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN', 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN', 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'PER_CMEAN', 'TUNDISHWEIGHTMEAN', 'SEG9+10+11+12TOPSECONDARYCOOLINGWATERFLOWACTUALVARIANCE', 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN', 'SEG5+6TOPSECONDARYCOOLINGWATERFLOWSETPOINTMEAN', 'PER_MOMEAN', 'MOLDTAPERPERCENTMEAN', 'PER_NIMEAN', 'SEG13+14BOTTOMSECONDARYCOOLINGWATERFLOWSETPOINTMEAN', 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN', 'BROADFACE2WATERFLOWVARIANCE', 'NARROWFACE1WATERFLOWMEAN', 'TUNDISHTEMPERATUREMEAN', 'LIQUIDUSTEMPMEAN', 'PER_CRMEAN']
Best accuracy score: 0.6890756302521008


In [17]:
accuracy_store

[0.6470588235294118,
 0.6470588235294118,
 0.5042016806722689,
 0.6470588235294118,
 0.5126050420168067,
 0.453781512605042,
 0.6386554621848739,
 0.6554621848739496,
 0.6218487394957983,
 0.5714285714285714,
 0.453781512605042,
 0.6554621848739496,
 0.6722689075630253,
 0.5294117647058824,
 0.6218487394957983,
 0.6386554621848739,
 0.5546218487394958,
 0.5546218487394958,
 0.5882352941176471,
 0.6554621848739496,
 0.4957983193277311,
 0.5546218487394958,
 0.6890756302521008,
 0.6134453781512605,
 0.5966386554621849,
 0.5546218487394958,
 0.6302521008403361,
 0.6722689075630253,
 0.6470588235294118,
 0.6302521008403361,
 0.5882352941176471,
 0.5546218487394958,
 0.5714285714285714,
 0.6722689075630253,
 0.46218487394957986,
 0.5966386554621849,
 0.6050420168067226,
 0.6638655462184874,
 0.5462184873949579,
 0.453781512605042,
 0.5462184873949579,
 0.5882352941176471,
 0.5210084033613446,
 0.6050420168067226,
 0.6470588235294118,
 0.47058823529411764,
 0.6050420168067226,
 0.66386554621

In [18]:
%store accuracy_store
%store base_features
%store base_accuracy
%store optimal_features

Stored 'accuracy_store' (list)
Stored 'base_features' (list)
Stored 'base_accuracy' (float64)
Stored 'optimal_features' (list)


In [2]:
%store -r base_features
len(base_features)

23

In [19]:
len(base_features)
base_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'MOLDWIDTHTOPMEAN',
 'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'PER_CMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG9+10+11+12TOPSECONDARYCOOLINGWATERFLOWACTUALVARIANCE',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'SEG5+6TOPSECONDARYCOOLINGWATERFLOWSETPOINTMEAN',
 'PER_MOMEAN',
 'MOLDTAPERPERCENTMEAN',
 'PER_NIMEAN',
 'SEG13+14BOTTOMSECONDARYCOOLINGWATERFLOWSETPOINTMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN',
 'BROADFACE2WATERFLOWVARIANCE',
 'NARROWFACE1WATERFLOWMEAN',
 'TUNDISHTEMPERATUREMEAN',
 'LIQUIDUSTEMPMEAN',
 'PER_CRMEAN']

In [21]:
new_features=[]
for feature in remaining_features:
    # Add one feature to the feature set
    new_features = base_features + [feature]

In [22]:
new_features

['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'STEELLEVELINMOLD-SETMEAN',
 'MOLDWIDTHTOPMEAN',
 'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN',
 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
 'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
 'SEG9+10+11+12BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'PER_CMEAN',
 'TUNDISHWEIGHTMEAN',
 'SEG9+10+11+12TOPSECONDARYCOOLINGWATERFLOWACTUALVARIANCE',
 'SEG7+8BOTTOMSECONDARYCOOLINGWATERFLOWACTUALMEAN',
 'SEG5+6TOPSECONDARYCOOLINGWATERFLOWSETPOINTMEAN',
 'PER_MOMEAN',
 'MOLDTAPERPERCENTMEAN',
 'PER_NIMEAN',
 'SEG13+14BOTTOMSECONDARYCOOLINGWATERFLOWSETPOINTMEAN',
 'MOLDNARROWFACECOOLINGWATERINLETTEMPMEAN',
 'BROADFACE2WATERFLOWVARIANCE',
 'NARROWFACE1WATERFLOWMEAN',
 'TUNDISHTEMPERATUREMEAN',
 'LIQUIDUSTEMPMEAN',
 'PER_CRMEAN',
 'SPEEDMEAN']

In [23]:
#Creating a new data
new_data=random_forest_data.loc[:,new_features]
new_data = shuffle(new_data, random_state=42)
new_data.shape

(591, 24)

In [24]:
 #Splitting the new_data into train and test
new_total_length= new_data.shape[0]
new_train_size=int(new_data.shape[0]*0.8)
new_test_size=new_data.shape[0]-train_size
new_train_X=new_data.iloc[:,:][0:train_size].to_numpy().astype(nm.float32) #2D array
new_train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
new_train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array
new_test_X=new_data.iloc[:,:][train_size:].to_numpy().astype(nm.float32) #2D array
new_test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
new_test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

NameError: name 'common_data' is not defined

In [25]:
new_train_X.shape

(472, 24)

In [26]:
# Create a new model with the updated feature set
new_model=Sequential() 
new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer

In [ ]:
# Compile the new model
new_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
# Train the new model
new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=5,validation_data=(new_test_X,new_test_Y))

# Evaluate the new model
testPredict_probs_new=new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y,y_new_pred)
    

Epoch 1/150
95/95 [==============================] - 1s 3ms/step - loss: 9.5605 - accuracy: 0.5021 - val_loss: 5.3987 - val_accuracy: 0.5462
Epoch 2/150
95/95 [==============================] - 0s 2ms/step - loss: 5.1354 - accuracy: 0.5551 - val_loss: 5.0582 - val_accuracy: 0.6050
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 3.8603 - accuracy: 0.5805 - val_loss: 3.4512 - val_accuracy: 0.6303
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 3.0868 - accuracy: 0.6102 - val_loss: 3.7149 - val_accuracy: 0.5462
Epoch 5/150
95/95 [==============================] - 0s 2ms/step - loss: 3.5663 - accuracy: 0.5763 - val_loss: 2.6649 - val_accuracy: 0.6303
Epoch 6/150
95/95 [==============================] - 0s 2ms/step - loss: 3.0244 - accuracy: 0.6038 - val_loss: 6.8316 - val_accuracy: 0.5714
Epoch 7/150
95/95 [==============================] - 0s 2ms/step - loss: 3.2900 - accuracy: 0.5932 - val_loss: 2.5974 - val_accuracy: 0.6387
Epoch 8/150
9

In [ ]:
random_forest_data.iloc[:,:-1].shape[1]


In [17]:
import time

# Compile the new model
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Start the timer
start_time = time.time()

# Train the new model
new_model.fit(new_train_X, new_train_Y, epochs=150, batch_size=5, validation_data=(new_test_X, new_test_Y))

# Stop the timer and calculate the elapsed time
end_time = time.time()
training_time = end_time - start_time

# Evaluate the new model
testPredict_probs_new = new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y, y_new_pred)

# Print the training time
print("Training Time: {:.2f} seconds".format(training_time))


Epoch 1/150
95/95 [==============================] - 1s 4ms/step - loss: 0.9540 - accuracy: 0.5339 - val_loss: 0.9819 - val_accuracy: 0.4118
Epoch 2/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8895 - accuracy: 0.4788 - val_loss: 0.8797 - val_accuracy: 0.4118
Epoch 3/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8629 - accuracy: 0.4852 - val_loss: 0.8526 - val_accuracy: 0.4202
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8230 - accuracy: 0.4979 - val_loss: 0.8514 - val_accuracy: 0.4118
Epoch 5/150
95/95 [==============================] - 0s 2ms/step - loss: 0.8401 - accuracy: 0.4703 - val_loss: 0.7900 - val_accuracy: 0.4286
Epoch 6/150
95/95 [==============================] - 0s 3ms/step - loss: 0.7832 - accuracy: 0.5318 - val_loss: 0.8835 - val_accuracy: 0.4370
Epoch 7/150
95/95 [==============================] - 0s 3ms/step - loss: 0.8159 - accuracy: 0.4746 - val_loss: 0.7572 - val_accuracy: 0.5378
Epoch 8/150
9

Epoch 59/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7085 - accuracy: 0.5699 - val_loss: 0.7057 - val_accuracy: 0.5378
Epoch 60/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7106 - accuracy: 0.5763 - val_loss: 0.7137 - val_accuracy: 0.5882
Epoch 61/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7261 - accuracy: 0.5805 - val_loss: 0.7059 - val_accuracy: 0.5546
Epoch 62/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7011 - accuracy: 0.5720 - val_loss: 0.6995 - val_accuracy: 0.5546
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7090 - accuracy: 0.5678 - val_loss: 0.6911 - val_accuracy: 0.5798
Epoch 64/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7293 - accuracy: 0.5297 - val_loss: 0.7111 - val_accuracy: 0.5378
Epoch 65/150
95/95 [==============================] - 0s 2ms/step - loss: 0.7450 - accuracy: 0.5636 - val_loss: 0.7623 - val_accuracy: 0.5042
Epoch 

In [19]:
a=[1]
a.append(2)

In [20]:
a

[1, 2]

In [30]:
a=0.69
b=0.5

In [31]:
print(a>b)

True


In [32]:
print(b>a)

False
